In [33]:
from api_key import API_KEY
import pandas as pd
import numpy as np
import requests
import json
import operator
import matplotlib.pyplot as plt

class Game:
    home: str
    away: str
    gameStr: str
    openLine: float # always in reference to home team
    curLine: float
    diff: float
    

year = 2021
seasonType = "regular"
week = input("Enter week number: ")

parameters = {"year":year, "week":week}
response = requests.get("https://api.collegefootballdata.com/lines", headers={'Authorization': API_KEY}, params = parameters)
lineResponse = response.json()
gameList = []

for i in range(len(lineResponse)):
    game = Game()
    game.home = lineResponse[i]['homeTeam']
    game.away = lineResponse[i]['awayTeam']
    game.gameStr = game.away + " @ " + game.home
    for j in range(len(lineResponse[i]['lines'])):
        if lineResponse[i]['lines'][j]['provider'] == 'Bovada':
            if lineResponse[i]['lines'][j]['spreadOpen'] == None: continue
            game.openLine = float(lineResponse[i]['lines'][j]['spreadOpen'])
            if lineResponse[i]['lines'][j]['spread'] == None: continue
            game.curLine = float(lineResponse[i]['lines'][j]['spread'])
            game.diff = abs(game.openLine - game.curLine)
            #print(game.gameStr)
            #print("Opening Line: " + str(game.home) + " " + str(game.openLine))
            #print("Current Line: " + game.home + " " + str(game.curLine))
            #print("Line Movement: " + str(game.diff))
            gameList.append(game)
gameList.sort(key=operator.attrgetter('diff'), reverse=True)

for i in range(len(gameList)):
    print(str(i + 1) + ".) " +  gameList[i].gameStr)
    print("Opening Line: %s %+d" % (gameList[i].home,  gameList[i].openLine))
    print("Current Line: %s %+d" % (gameList[i].home, gameList[i].curLine))
    print("Line Movement: %d\n" % gameList[i].diff)


1.) Georgia @ Alabama
Opening Line: Alabama +4
Current Line: Alabama +6
Line Movement: 2

2.) Western Kentucky @ UT San Antonio
Opening Line: UT San Antonio +1
Current Line: UT San Antonio +3
Line Movement: 2

3.) USC @ California
Opening Line: California -2
Current Line: California -4
Line Movement: 2

4.) Utah State @ San Diego State
Opening Line: San Diego State -4
Current Line: San Diego State -6
Line Movement: 2

5.) Kent State @ Northern Illinois
Opening Line: Northern Illinois +2
Current Line: Northern Illinois +4
Line Movement: 2

6.) Michigan @ Iowa
Opening Line: Iowa -10
Current Line: Iowa -12
Line Movement: 1

7.) Houston @ Cincinnati
Opening Line: Cincinnati -10
Current Line: Cincinnati -10
Line Movement: 0

8.) Appalachian State @ Louisiana
Opening Line: Louisiana +2
Current Line: Louisiana +2
Line Movement: 0

9.) Baylor @ Oklahoma State
Opening Line: Oklahoma State -6
Current Line: Oklahoma State -7
Line Movement: 0

10.) Pittsburgh @ Wake Forest
Opening Line: Wake Fores

In [55]:
from api_key import API_KEY
import pandas as pd
import numpy as np
import scipy
import requests
import json
import operator
import math
import random
import statistics
import matplotlib.pyplot as plt
import cfbd
from pprint import pprint

# Configure API key authorization: ApiKeyAuth
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = API_KEY

# create an instance of the API class
api = cfbd.GamesApi(cfbd.ApiClient(configuration))
lines_api = cfbd.BettingApi(cfbd.ApiClient(configuration))
year = 2021 # int | Year/season filter for games
week = 56 # int | Week filter (optional)
season_type = 'regular' # str | Season type filter (regular or postseason) (optional) (default to regular)
team = 'team_example' # str | Team filter (optional)
conference = 'conference_example' # str | Conference abbreviation filter (optional)
category = 'category_example' # str | Category filter (e.g defensive) (optional)
game_id = 56 # int | Game id filter (optional)

ptsFor = {}
ptsAgainst = {}
nGames = {}
stdDev = {}

try:
    # Player game stats
    api_response = api.get_games(year)
    #pprint(api_response)
except ApiException as e:
    print("Exception when calling GamesApi->get_player_game_stats: %s\n" % e)
    
for game in range(len(api_response)):
    if api_response[game].home_team in stdDev:
        if api_response[game].home_points != None:
            stdDev[api_response[game].home_team].append(api_response[game].home_points)
        else: 
            stdDev[api_response[game].home_team].append(0)
    else:
        if api_response[game].home_points != None:
            stdDev[api_response[game].home_team] = []
            stdDev[api_response[game].home_team].append(api_response[game].home_points)
        else: 
            stdDev[api_response[game].home_team] = []
            stdDev.append(0)
    if api_response[game].away_team in stdDev:
        if api_response[game].away_points != None:
            stdDev[api_response[game].away_team].append(api_response[game].away_points)
        else: 
            stdDev[api_response[game].away_team].append(0)
    else:
        if api_response[game].away_points != None:
            stdDev[api_response[game].away_team] = []
            stdDev[api_response[game].away_team].append(api_response[game].away_points)
        else: 
            stdDev[api_response[game].away_team] = []
            stdDev.append(0)
                   
    if api_response[game].home_team in nGames:
        nGames[api_response[game].home_team] += 1
    else:
        nGames[api_response[game].home_team] = 1
    if api_response[game].away_team in nGames:
        nGames[api_response[game].away_team] += 1
    else:
        nGames[api_response[game].away_team] = 1
        
    if api_response[game].home_team in ptsFor:
        if api_response[game].home_points is None:
            ptsFor[api_response[game].home_team] += 0
        else:
            ptsFor[api_response[game].home_team] += api_response[game].home_points
    else:
        ptsFor[api_response[game].home_team] = api_response[game].home_points
    if api_response[game].away_team in ptsFor:
        if api_response[game].away_points is None:
            ptsFor[api_response[game].away_team] += 0
        else:
            ptsFor[api_response[game].away_team] += api_response[game].away_points
    else:
        ptsFor[api_response[game].away_team] = api_response[game].away_points
        
    if api_response[game].home_team in ptsAgainst:
        if api_response[game].away_points is None:
            ptsAgainst[api_response[game].home_team] += 0
        else:
            ptsAgainst[api_response[game].home_team] += api_response[game].away_points
    else:
        ptsAgainst[api_response[game].home_team] = api_response[game].away_points
    if api_response[game].away_team in ptsAgainst:
        if api_response[game].home_points is None:
            ptsAgainst[api_response[game].away_team] += 0
        else:
            ptsAgainst[api_response[game].away_team] += api_response[game].home_points
    else:
        ptsAgainst[api_response[game].away_team] = api_response[game].home_points
    
for key in stdDev:
    stdDev[key].pop()
# pprint(ptsFor)
# pprint(ptsAgainst)
# pprint(nGames)
# pprint(stdDev)

week = input("Enter week number: ")

try:
    api_response = api.get_games(year, week=int(week))
    # pprint(api_response)
except ApiException as e:
    print("Exception when calling GamesApi->get_player_game_stats: %s\n" % e)
    
overUnderWins = 0
overUnderLosses = 0
spreadWins = 0
spreadLosses = 0
avgSpreadWins = 0
avgSpreadLosses = 0

for game in range(len(api_response)):
    finalHomePts = api_response[game].home_points
    finalAwayPts = api_response[game].away_points
    homeAvgPts = ptsFor[api_response[game].home_team] / nGames[api_response[game].home_team]
    homeAvgPtsAgainst = ptsAgainst[api_response[game].home_team] / nGames[api_response[game].home_team]
    homeAdjustedPts = math.sqrt(homeAvgPts * homeAvgPtsAgainst)
    if stdDev[api_response[game].home_team] != []:
        homeStdDev = statistics.pstdev(stdDev[api_response[game].home_team])
    
    # print("%s AvgPts = %-d AvgPtsAgainst = %-d adjusted pts = %-d" % (api_response[game].home_team, homeAvgPts, homeAvgPtsAgainst, homeAdjustedPts))
    
    awayAvgPts = ptsFor[api_response[game].away_team] / nGames[api_response[game].away_team]
    awayAvgPtsAgainst = ptsAgainst[api_response[game].away_team] / nGames[api_response[game].away_team]
    awayAdjustedPts = math.sqrt(awayAvgPts * awayAvgPtsAgainst)
    if stdDev[api_response[game].away_team] != []:
        awayStdDev = statistics.pstdev(stdDev[api_response[game].away_team])
   
    homePredPoints = 0
    awayPredPoints = 0
    for i in range(10000):
        if homeStdDev != 0 and awayStdDev != 0:
            homePredPoints += statistics.NormalDist(mu=homeAdjustedPts, sigma=homeStdDev).inv_cdf(random.random())
            awayPredPoints += statistics.NormalDist(mu=awayAdjustedPts, sigma=awayStdDev).inv_cdf(random.random())
    
    homePredPoints = (homePredPoints + 2.5)/10000
    awayPredPoints = awayPredPoints/10000
    print("OUTCOME IF SCORE AVERAGE TAKEN OF ALL 10,000 SIMS")
    print("%s %.2d - %d.2 %s  -  total = %d.2" % (api_response[game].home_team.upper(), homePredPoints, awayPredPoints, api_response[game].away_team, (homePredPoints+awayPredPoints)))
    lines_response = lines_api.get_lines(week=int(week), year=year, home=api_response[game].home_team, away=api_response[game].away_team)
    print("TOTAL IS AN AVERAGE OF PTS FROM ALL 10,000 SIMS")
    print("Total %d - Predicted %d.2 -> diff %d.2" % (float(lines_response[0].lines[0]['overUnder']), (homePredPoints+awayPredPoints), abs(float(lines_response[0].lines[0]['overUnder']) - (homePredPoints+awayPredPoints))))

    #if the model picked the over
    if (homePredPoints + awayPredPoints) > float(lines_response[0].lines[0]['overUnder']):
        #if the over actually hit
        if (finalHomePts + finalAwayPts) >= float(lines_response[0].lines[0]['overUnder']):
            overUnderWins += 1
            print("over/under WIN")
        else:
            overUnderLosses += 1
            print("over/under LOSS")
    #if the model picked the under
    if (homePredPoints + awayPredPoints) < float(lines_response[0].lines[0]['overUnder']):
        #if the under actually hit
        if (finalHomePts + finalAwayPts) <= float(lines_response[0].lines[0]['overUnder']):
            overUnderWins += 1
            print("over/under WIN")
        else:
            overUnderLosses += 1
            print("over/under LOSS")


    homePredPoints = 0
    awayPredPoints = 0
    homeWins = 0
    awayWins = 0
    homeAvgMargin = 0
    awayAvgMargin =0
    actualMargin = 0
    relMargin = 0

    for i in range(10000):
        if homeStdDev != 0 and awayStdDev != 0:
            homePredPoints = (statistics.NormalDist(mu=homeAdjustedPts, sigma=homeStdDev).inv_cdf(random.random()) + 2.5)
            awayPredPoints = statistics.NormalDist(mu=awayAdjustedPts, sigma=awayStdDev).inv_cdf(random.random()) 
            if(homePredPoints > awayPredPoints):
                homeWins += 1
                homeAvgMargin += (homePredPoints - awayPredPoints)
            else:
                awayWins += 1
                awayAvgMargin += (awayPredPoints - homePredPoints)
    if(homeWins != 0): 
        homeAvgMargin = homeAvgMargin / homeWins
    else:
        homeAvgMargin = 0
    if (awayWins != 0):
        awayAvgMargin = awayAvgMargin / awayWins
    else:
        awayAvgMargin = 0
    
    relMargin = (0-awayAvgMargin)
    homeWinPct = (homeWins / 10000) * 100
    awayWinPct = (awayWins / 10000) * 100

    if lines_response[0].lines[0]['spread'] is None:
        continue
    spread = float(lines_response[0].lines[0]['spread'])

    pickHome = False
    pickAway = False

    print(lines_response[0].lines[0]['formattedSpread'])
    print("spread = %.2f" % float(lines_response[0].lines[0]['spread']))
    print("over/uner = %.2f" % float(lines_response[0].lines[0]['overUnder']))
    print("OUTCOME IF EACH SIM PROCESSED INDIVIDUALLY")
    if(homeWinPct > awayWinPct):
        print("Home Team %s wins %.2f percent of the sims by an avg of %.2f pts" % (api_response[game].home_team.upper(), homeWinPct, homeAvgMargin))
        if(homeAvgMargin+spread > 0):
            pickHome = True
            print("PICK HOME")
        else:
            pickAway = True
            print("PICK AWAY")
    else:
        print("Away team %s wins %.2f percent of the sims by an avg of %.2f pts" % (api_response[game].away_team.upper(), awayWinPct, awayAvgMargin))
        print("relative margin = %.2f" % relMargin)
        if(awayAvgMargin+(0-spread) > 0):
            pickAway = True
            print("PICK AWAY")
        else:
            pickHome = True
            print("PICK HOME")

    if pickHome == True:
        if (finalHomePts+spread) > finalAwayPts:
            avgSpreadWins += 1
            print("AvgSpread WIN")
        else:
            avgSpreadLosses += 1
            print("AvgSpread LOSS")
    else:
        if (finalAwayPts+(0-spread) > finalHomePts):
            avgSpreadWins += 1
            print("AvgSpread WIN")
        else:
            avgSpreadLosses += 1
            print("AvgSpread LOSS")

    print("Actual Score: HOME %.2f - %.2f Away - diff -> %.2f\n" % (finalHomePts, finalAwayPts, abs(finalHomePts - finalAwayPts)))

spreadWinPct = (avgSpreadWins / (avgSpreadWins+avgSpreadLosses)) * 100
overUnderWinPct = (overUnderWins/(overUnderWins+overUnderLosses)) * 100

print("Spread win percentage %.2f" % spreadWinPct) 
print("Over Under win percentage %.2f" % overUnderWinPct)

OUTCOME IF SCORE AVERAGE TAKEN OF ALL 10,000 SIMS
UT SAN ANTONIO 29 - 35.2 Western Kentucky  -  total = 64.2
TOTAL IS AN AVERAGE OF PTS FROM ALL 10,000 SIMS
Total 74 - Predicted 64.2 -> diff 9.2
over/under LOSS
Western Kentucky -3.5
spread = 3.50
over/uner = 74.50
OUTCOME IF EACH SIM PROCESSED INDIVIDUALLY
Away team WESTERN KENTUCKY wins 57.38 percent of the sims by an avg of 11.79 pts
relative margin = -11.79
PICK AWAY
AvgSpread LOSS
Actual Score: HOME 49.00 - 41.00 Away - diff -> 8.00

OUTCOME IF SCORE AVERAGE TAKEN OF ALL 10,000 SIMS
UTAH 26 - 28.2 Oregon  -  total = 55.2
TOTAL IS AN AVERAGE OF PTS FROM ALL 10,000 SIMS
Total 57 - Predicted 55.2 -> diff 2.2
over/under WIN
Utah -3
spread = -3.00
over/uner = 57.50
OUTCOME IF EACH SIM PROCESSED INDIVIDUALLY
Home Team UTAH wins 53.23 percent of the sims by an avg of 12.26 pts
PICK HOME
AvgSpread WIN
Actual Score: HOME 38.00 - 10.00 Away - diff -> 28.00

OUTCOME IF SCORE AVERAGE TAKEN OF ALL 10,000 SIMS
NORTHERN ILLINOIS 32 - 34.2 Kent St